<b>Steps:</b>

1. Use the New York Times API to fetch the news articles based on the section names
2. Iterate and save all the urls
3. Hit the urls seperately and fetch the html content
4. Use the BeautifulSoap to parse the html
5. Get the content of the url and save the whole content in a single file per article

https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [ ]:
import json
import sys
import requests
from bs4 import BeautifulSoup
import re
import os
import time

In [ ]:
!{sys.executable} -m pip install beautifulsoup4

In [ ]:
def remove_tag(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [ ]:
def get_data(section_name, folder_name, file_count):
    
    web_url = []
    
    # Search for first 200 pages of New York Times for the given section name
    for i in range(0,200): 
        params = {
        'api-key': '', # ToDo: Provide your API Key
        'fq': 'news_desk:("'+section_name+'")',
        'page' : str(i),
        'fl': "web_url",
        'sort': "newest"
        }
        time.sleep(1)
        nytimes_response = requests.get(url='https://api.nytimes.com/svc/search/v2/articlesearch.json', params= params)
        str_nytimes_response = nytimes_response.content.decode("utf-8") 
        json_nytimes_response = json.loads(str_nytimes_response)
        if(json_nytimes_response['response']['docs'] != []):
            for i in json_nytimes_response['response']['docs']:
                web_url.append(i['web_url'])
            print(len(web_url))
        else :
            break
            
    for url in web_url:
        try:
            url_response = requests.get(url)
            soup = BeautifulSoup(url_response.content, 'html.parser')
            story = soup.findAll("p", class_="story-content")
            e2kc3sl0 = soup.findAll("p", class_="e2kc3sl0")
            first_draft = soup.findAll("p", class_="story-body-text")
            
            web_text = ''
            for text in story:
                temp = remove_tag(str(text))
                web_text = web_text+" "+temp

            if web_text == '':
                for e in e2kc3sl0:
                    t = remove_tag(str(e))
                    web_text = web_text+" "+t
                
            if web_text == '':
                for draft in first_draft:
                    td = remove_tag(str(draft))
                    web_text = web_text+" "+td
            

            file_count = file_count + 1;
            file_name = section_name+"_"+str(file_count)+".txt"

            print("File Name: "+file_name)
            print("url: "+url)

            filepath = os.path.join(os.getcwd()+'/'+folder_name, file_name)
            if not os.path.exists(os.getcwd()+'/'+folder_name):
                os.makedirs(os.getcwd()+'/'+folder_name)
            f = open(filepath, "w")
            f.write(web_text)
            f.close()
        except: # Any other exception
            pass
    return web_url

In [ ]:
urls = get_data("","",0) # ToDo: Add your section name, folder name